# Outfitters - All Sections

## Retrieving everything

In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import urllib.request
import pandas as pd
import time
import requests
import json
import os

In [2]:
SectionNames = []
sectionlinks = []
faultylinks = [[],[],[],[],[],[]]
pages = [[],[],[],[],[],[]]
productlinks = [[],[],[],[],[],[]]
all_rows = [[],[],[],[],[],[]]

In [3]:
faultylinks2 = [[],[],[],[],[],[]]
faultylinks3 = [[],[],[],[],[],[]]

In [4]:
u = requests.get('https://outfitters.com.pk').text
base_url = 'https://outfitters.com.pk'
page_soup = soup(u, "lxml")
for h in page_soup.findAll('div', {'class':['right-child', 'left-child']}):
    SectionNames.append(os.path.basename(h.a['href']))
    sectionlinks.append(base_url + h.a['href'] + '?page=1&view=all')

In [5]:
sectionlinks

['https://outfitters.com.pk/collections/men?page=1&view=all',
 'https://outfitters.com.pk/collections/women?page=1&view=all',
 'https://outfitters.com.pk/collections/boys?page=1&view=all',
 'https://outfitters.com.pk/collections/girls?page=1&view=all',
 'https://outfitters.com.pk/collections/outwear-jackets?page=1&view=all']

In [6]:
%%time
#Scraping Product Links
n = -1
for link in sectionlinks:
    n += 1
    page_soup = soup(requests.get(link).text, 'lxml')
    products = page_soup.findAll('div', {'class':'product-image'})
    for k in products:
        productlinks[n].append(base_url + k.a['href'])
    pages = page_soup.find('div', {'class':'infinite-scrolling'})
    if pages is not None:
        while (pages is not None):
            link2 = base_url + pages.a['href']
            page_soup = soup(requests.get(link2).text, 'lxml')
            products = page_soup.findAll('div', {'class':'product-image'})
            for k in products:
                productlinks[n].append(base_url + k.a['href'])
            pages = page_soup.find('div', {'class':'infinite-scrolling'})
            print(link2 + '......CoMpLeTeD')

https://outfitters.com.pk/collections/men?page=2&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/men?page=3&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/men?page=4&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/women?page=2&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/women?page=3&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/boys?page=2&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/boys?page=3&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/boys?page=4&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/boys?page=5&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/boys?page=6&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/girls?page=2&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/girls?page=3&view=all......CoMpLeTeD
https://outfitters.com.pk/collections/girls?page=4&view=all......CoMpLeTeD
https://outfitters.com.pk/collection

In [3]:
#with open('productlinksALL.txt', 'w') as filehandle:
    #json.dump(productlinks, filehandle)
    
with open('productlinksALL.txt', 'r') as filehandle:
    productlinks = json.load(filehandle)

In [7]:
colors = []

In [8]:
%%time
#Scraping ALL product
i = -1
for b in productlinks:
    i += 1
    h=0
    path = 'G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Outfitters/' + SectionNames[i]
    if not os.path.exists(path):
        os.mkdir(path)
    print("Section " + str(i) + " In Progress...")
    for link in b:
        try:
            s = requests.get(link).text
            page_soup = soup(s, 'lxml')
            name = None
            prices = None
            oldprice= newprice= color= all_sizes= sizes= Description = None
            name = page_soup.find("h2", {"itemprop":"name"}).text.strip()
            #print(name)
            prices = page_soup.find("div", {'class':'prices'})('span')
            oldprice = prices[0].text.strip()
            newprice = prices[1].text.strip()
            if (newprice == oldprice):
                newprice = prices[2].text.strip()
            op = int(oldprice.replace("PKR", "").replace(",", "").strip())
            np = int(newprice.replace("PKR", "").replace(",", "").strip())
            discount =str(int((op-np)/op * 100)) + "%"
            #print(prices)
            try:
                color = page_soup.find("label", {"data-title":name})
                colors.append(color['data-variant'].strip())
            except:
                color = page_soup.find("div", {"class":'tooltip'})
                colors.append(color.text.strip())

            #print(colors)
            sizes = page_soup.find("div", {'class':'wrapper text-center Size'})('div')
            #print(sizes)
            all_sizes = []
            for size in sizes[1:]:
                all_sizes.append(size['data-value'])

            all_sizes = ','.join(all_sizes)
            D = page_soup.find('div', {'class':'lang1'}).text
            #print(D)
            Desc = D.find('Description')
            Det = D.find('Detail')
            SizenFit = D.find('Fits')
            CompnCare = D.find('Care')
            if (SizenFit != -1 and (Desc != -1 or Det != -1)):
                if (Desc == -1):
                    Description = D[Desc+16:SizenFit-7].strip()
                else:
                    Description = D[Desc+12:SizenFit-7].strip()
            elif (SizenFit == -1 and CompnCare != -1):
                Description = D[Desc+12: CompnCare-14].strip()
            if CompnCare is not -1:
                care = D[CompnCare+5:].strip()
            
            row = {"Name":name, "Description":Description, "Composition & Care": care, "Old Price":oldprice, "New Price": newprice,"Discount":discount, "Size(s)": all_sizes, "Product Link":link}
            all_rows[i].append(row)
            #images:
            i = page_soup.find("ul", {'id':'more-view-carousel'})('li')
            m = -1  
            for a in i:
                m += 1
                filename = str(h) + '.' + str(m) + '.jpg'
                urllib.request.urlretrieve("https:" + a.a['data-image'], os.path.join(path, filename))
        except:
            faultylinks[i].append(link)
            print(link)

Section 0 Completed
https://outfitters.com.pk/collections/men/products/mtf491020-blk
https://outfitters.com.pk/collections/men/products/maf491233-yel
https://outfitters.com.pk/collections/men/products/mtf491060-hgr
https://outfitters.com.pk/collections/men/products/mtf491020-bur
https://outfitters.com.pk/collections/men/products/mtf491084-blk
https://outfitters.com.pk/collections/men/products/mtf491083-blk
https://outfitters.com.pk/collections/men/products/mtf491083tel
https://outfitters.com.pk/collections/men/products/mtf491029-gry
https://outfitters.com.pk/collections/men/products/mtf491063-bkw
Section 1 Completed
https://outfitters.com.pk/collections/women/products/wtf492018-plm
https://outfitters.com.pk/collections/women/products/wtc492084-pnr
https://outfitters.com.pk/collections/women/products/waf492214-yel
https://outfitters.com.pk/collections/women/products/wtc492062-tel
https://outfitters.com.pk/collections/women/products/wtc492062-blk
https://outfitters.com.pk/collections/wom

In [20]:
link = 'https://outfitters.com.pk/collections/men/products/mtf491067rdb?variant=31672903663696'
s = requests.get(link).text
page_soup = soup(s, 'lxml')
h=0
name = None
prices = None
discount = None
oldprice= newprice= color= all_sizes= sizes= Description = None
name = page_soup.find("h2", {"itemprop":"name"}).text.strip()
#print(name)
prices = page_soup.find("div", {'class':'prices'})('span')
oldprice = prices[0].text.strip()
newprice = prices[1].text.strip()
if (newprice == oldprice):
    newprice = prices[2].text.strip()
op = int(oldprice.replace("PKR", "").replace(",", "").strip())
np = int(newprice.replace("PKR", "").replace(",", "").strip())
discount =str(int((op-np)/op * 100)) + "%"
#print(prices)
#color = page_soup.find("label", {"data-title":name})
color = page_soup.find("div", {"class":'tooltip'})
print(color.text)
#colors.append(color['data-variant'].strip())
#print(colors)
sizes = page_soup.find("div", {'class':'wrapper text-center Size'})('div')
#print(sizes)
all_sizes = []
for size in sizes[1:]:
    all_sizes.append(size['data-value'])

all_sizes = ','.join(all_sizes)
D = page_soup.find('div', {'class':'lang1'}).text
#print(D)
Desc = D.find('Description')
Det = D.find('Detail')
SizenFit = D.find('Fits')
CompnCare = D.find('Care')
if (SizenFit != -1 and (Desc != -1 or Det != -1)):
    if (Desc == -1):
        Description = D[Desc+16:SizenFit-7].strip()
    else:
        Description = D[Desc+12:SizenFit-7].strip()
elif (SizenFit == -1 and CompnCare != -1):
    Description = D[Desc+12: CompnCare-14].strip()

care = D[CompnCare+5:].strip()
#images:
i = page_soup.find("ul", {'id':'more-view-carousel'})('li')
m = -1  
for a in i:
    m += 1
    filename = str(h) + '.' + str(m) + '.jpg'
    urllib.request.urlretrieve("https:" + a.a['data-image'], os.path.join(path, filename))
#print(Description)
row = {"Name":name, "Description":Description, "Composition & Care": care, "Old Price":oldprice, "New Price": newprice,"Discount":discount, "Size(s)": all_sizes, "Product Link":link}
print(np, op, discount)

RED BLK
1490 2090 28%


In [10]:
%%time
for i in range(len(sectionlinks)):
    products = pd.DataFrame(all_rows[i], columns = ["Name", "Description", "Composition & Care", "Old Price", "New Price","Discount", "Size(s)","Product Link"])
    path = 'G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Outfitters/' + SectionNames[i]
    if not os.path.exists(path):
        os.mkdir(path)
    print(path)
    products.to_csv(os.path.join(path, SectionNames[i] + "_all.csv"), encoding='utf-8')

G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Outfitters/men
G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Outfitters/women
G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Outfitters/boys
G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Outfitters/girls
G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Outfitters/outwear-jackets
Wall time: 115 ms
